## Objective
- 게임설명이 비슷한 게임들을 추천하는 시스템 제작

### Data Load

In [10]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier


import tensorflow as tf
from sklearn.metrics.pairwise import linear_kernel
from sentence_transformers import SentenceTransformer

- Sentence-BERT 모델은 BERT를 기반으로 sentence embedding vector를 생성하는 모델.
	BERT 모델은 문장 내 모든 단어들을 참고한 문맥을 가진 임베딩 벡터를 만드는 모델이며 문장의 의미를 추출하는데 강점임.

In [12]:
# 라이브러리
'''

SentenceTransformer는 HuggingFace 라이브러리, 모델을 다운받아 사용하기 위한 라이브러리임.
linear_kernel는 여러 문장과 문장 사이의 코사인 유사도를 측정할 때 사용하는 라이브러리임.
pandas는 데이터를 DataFrame과 Series를 만들거나 전처리하는데 필요한 라이브러리임.
numpy는 텍스트를 문장 임베딩 벡터로 생성한 값을 저장, 불러오기 위한 라이브러리임.

'''



'\n\nSentenceTransformer는 HuggingFace 라이브러리, 모델을 다운받아 사용하기 위한 라이브러리임.\nlinear_kernel는 여러 문장과 문장 사이의 코사인 유사도를 측정할 때 사용하는 라이브러리임.\npandas는 데이터를 DataFrame과 Series를 만들거나 전처리하는데 필요한 라이브러리임.\nnumpy는 텍스트를 문장 임베딩 벡터로 생성한 값을 저장, 불러오기 위한 라이브러리임.\n\n'

In [19]:
file_path = 'D:\Code\DataSets/games.csv'

df_games = pd.read_csv(file_path)

print(df_games)

         AppID                                    Name  Release date  \
0        20200                        Galactic Bowling  Oct 21, 2008   
1       655370                            Train Bandit  Oct 12, 2017   
2      1732930                            Jolt Project  Nov 17, 2021   
3      1355720                                Henosis™  Jul 23, 2020   
4      1139950                   Two Weeks in Painland   Feb 3, 2020   
...        ...                                     ...           ...   
71711  2159120                                    SUR5   Apr 1, 2023   
71712  2125160                           Prison Life 2  Mar 30, 2023   
71713  2318860                  Architecture Zeitgeist  Mar 27, 2023   
71714  2180330  Girl's Way to Survive 2: Bloody Museum   Apr 2, 2023   
71715  2352890                            Hentai Ariel  Mar 30, 2023   

      Estimated owners  Peak CCU  Required age  Price  DLC count  \
0            0 - 20000         0             0  19.99          0   

### Pre-processing

In [20]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71716 entries, 0 to 71715
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       71716 non-null  int64  
 1   Name                        71715 non-null  object 
 2   Release date                71716 non-null  object 
 3   Estimated owners            71716 non-null  object 
 4   Peak CCU                    71716 non-null  int64  
 5   Required age                71716 non-null  int64  
 6   Price                       71716 non-null  float64
 7   DLC count                   71716 non-null  int64  
 8   About the game              69280 non-null  object 
 9   Supported languages         71716 non-null  object 
 10  Full audio languages        71716 non-null  object 
 11  Reviews                     9167 non-null   object 
 12  Header image                71716 non-null  object 
 13  Website                     350

In [21]:
data = df_games.loc[:,['Name','About the game']]

print(data)

                                         Name  \
0                            Galactic Bowling   
1                                Train Bandit   
2                                Jolt Project   
3                                    Henosis™   
4                       Two Weeks in Painland   
...                                       ...   
71711                                    SUR5   
71712                           Prison Life 2   
71713                  Architecture Zeitgeist   
71714  Girl's Way to Survive 2: Bloody Museum   
71715                            Hentai Ariel   

                                          About the game  
0      Galactic Bowling is an exaggerated and stylize...  
1      THE LAW!! Looks to be a showdown atop a train....  
2      Jolt Project: The army now has a new robotics ...  
3      HENOSIS™ is a mysterious 2D Platform Puzzler w...  
4      ABOUT THE GAME Play as a hacker who has arrang...  
...                                                  ... 

### 결측치 확인 및 제거

In [27]:
print(f'각 컬럼별 누락 삭제 전 데이터의 수 : \n{data.isnull().sum()}')

# 누락 데이터 존재.

print('*'*80)

cleaned_df = data.dropna()
# 제거할 게 많을 땐 컬럼지정해서 dropna(subset)

print(f'각 컬럼별 누락 삭제 후 데이터의 수 : \n{cleaned_df.isnull().sum()}')
